In [12]:
!pip install setuptools==69.5.1

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from finn.util.visualization import showSrc, showInNetron
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

<h1>Model</h1>

In [14]:
#hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 20
bits = 8

<h2>Dataset</h2>

In [15]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [16]:
import brevitas
import brevitas.nn as qnn
from brevitas.export import export_qonnx
from brevitas.quant import Int8Bias

class QuantNeuralNet(nn.Module):
    def __init__(self):
        super(QuantNeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.quant = qnn.QuantIdentity(bit_width=bits, return_quant_tensor=True)
        self.linear1 = qnn.QuantLinear(in_features=28*28, out_features=512, bias=True, weight_bit_width=bits, bias_quant=Int8Bias)
        self.relu1 = qnn.QuantReLU(bit_width=bits, return_quant_tensor=True)
        self.linear2 = qnn.QuantLinear(in_features=512, out_features=512, bias=True, weigth_bit_width=bits, bias_quant=Int8Bias)
        self.relu2 = qnn.QuantReLU(bit_width=bits, return_quant_tensor=True)
        self.linear3 = qnn.QuantLinear(in_features=512, out_features=10, bias=True, weight_bit_width=bits, bias_quant=Int8Bias)

    def forward(self, x):
        x = self.flatten(x)
        x = self.quant(x)
        out = self.linear1(x)
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        logits = self.linear3(out)
        return logits

model = QuantNeuralNet()

ImportError: cannot import name 'packaging' from 'pkg_resources' (/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

<h3>Train</h3>

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

<h3>Export to qonnx</h3>

In [ ]:
export_qonnx_path = "mnist_model_q.onnx"
input_shape = (1, 28, 28)
export_qonnx(model, torch.randn(input_shape), export_qonnx_path)

In [ ]:
showInNetron(export_qonnx_path)

<h1>Import trained PyTorch Model with ONNX<h1>

Cleanup transformation on QONNX Model

In [ ]:
from qonnx.util.cleanup import cleanup

export_onnx_path_cleaned = "mnist_model_q_clean.onnx"
cleanup(export_qonnx_path, out_file=export_onnx_path_cleaned)

Import into FINN

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe
import onnx.numpy_helper as nph
import onnx.helper as onh
import onnx

sample_idx = torch.randint(len(test_data), size=(1,)).item()
input_img, label = test_data[sample_idx]


model = ModelWrapper(export_onnx_path_cleaned)
input_dict = {"global_in": input_img.numpy()}
output_dict = oxe.execute_onnx(model, input_dict)

produced_qonnx = output_dict[list(output_dict.keys())[0]]

produced_qonnx

In [ ]:
import matplotlib.pyplot as plt

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
plt.title(labels_map[produced_qonnx.argmax()])
plt.imshow(input_img.squeeze(), cmap="gray")
plt.show()

<h2> Convert using QONNXtoFINN </h2>

In [ ]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.core.datatype import DataType

model.set_tensor_datatype(model.graph.input[0].name, DataType[""])
model = model.transform(ConvertQONNXtoFINN())

export_onnx_path_converted = "mnist_model_q_converted.onnx"
model.save(export_onnx_path_converted)

<h1> Launch a Build: Only Estimate Reports </h1>

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

In [ ]:
model_file = "mnist_model_q_converted.onnx"

estimates_output_dir = "output_estimates_only"


#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7a35ticsg324-1L",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

In [ ]:
assert os.path.exists(estimates_output_dir + "/report/estimate_network_performance.json")

In [ ]:
! ls {estimates_output_dir}

In [ ]:
! ls {estimates_output_dir}/report

In [ ]:
! cat {estimates_output_dir}/report/estimate_network_performance.json

In [ ]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [ ]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

In [ ]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

<h2> Build Stitched IP </h2>

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "mnist_model_q_converted.onnx"

rtlsim_output_dir = "output_ipstitch_rtlsim"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    shutil.rmtree(rtlsim_output_dir)
    print("Previous run results deleted!")

cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7a35ticsg324-1L",
    generate_outputs=[
        build_cfg.DataflowOutputType.STITCHED_IP,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)

In [ ]:
showInNetron("cybersecurity/cybsec-mlp-ready.onnx")